In [ ]:
from google.colab import files
uploaded = files.upload("Preprocessed_jobs.csv")

import io
import pandas as pd

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

  # Assuming the uploaded file is a CSV
  df = pd.read_csv(io.BytesIO(uploaded[fn]))

  # Now you can work with the DataFrame 'df'
  print(df.head())

Saving Preprocessed_jobs.csv to Preprocessed_jobs.csv
User uploaded file "Preprocessed_jobs.csv" with length 5017328 bytes
   Unnamed: 0         Search Term  \
0           0  Transformer Models   
1           1  Transformer Models   
2           2  Transformer Models   
3           3  Transformer Models   
4           4  Transformer Models   

                                     Job Description  Line Break Count  \
0  About Us:\nRivian and Volkswagen Group Technol...                42   
1  Job Description\nAI Research Scientist, Founda...                68   
2  Valence has built the first-to-market AI nativ...                46   
3  Job Description\n\nIntel AI Lab is a research ...                44   
4  Company Overview:\nAbout SS8 Networks: As a le...                53   

                             Cleaned Job Description  
0  rivian and volkswagen group technologies is a ...  
1  ai research scientist, foundation models prima...  
2  valence has built the firsttomarket ai na

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

# Load the tokenizer and model
model_name = "jjzha/jobbert_skill_extraction"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Create a pipeline for token classification
nlp_skill_extraction = pipeline(
    "token-classification",
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple"
)

# Chunking function based on token length
def chunk_text(text, tokenizer, max_len=500):
    tokens = tokenizer(text, truncation=False, return_offsets_mapping=False, return_tensors="pt")["input_ids"][0]
    token_chunks = [tokens[i:i+max_len] for i in range(0, len(tokens), max_len)]
    return [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in token_chunks]

# Split into chunks
chunks = [chunk_text(desc, tokenizer) for desc in df['Cleaned Job Description'].tolist()]

# Flatten chunks and extract skills
all_skills = []
for chunk_list in chunks:
    for chunk in chunk_list:
        results = nlp_skill_extraction(chunk)
        all_skills.extend([r['word'] for r in results])

# Deduplicate
unique_skills_CDS = list(set(all_skills))

# Output
print(unique_skills_CDS)

Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


['##s research efforts', 'highly complex issues escalated by direct reports. responsibilities organizes and facilitates', 'responsible ai usage', 'and support forest', 'conferences, and', 'large multimodal foundation models', 'systems that balance scalability, availability, and', 'awareness of the hotline call system', ', and optimize ai agents for performance, robustness, and scalability', 'detect', ', maintain, analyze, and download various reporting spreadsheets', 'business requirements', 'and analyze user needs', 'projects simultaneously', 'accounts', 'the work plan', 'and improve workflows', 'technical information to nontechnical audiences', 'll', 'machine learning and optimization techniques', 'collaboration', 'alignment with strategic priorities', 'transformational leadership', 'decisions on system architecture', 'ai technologies', 'and manage large scale text, video and motion datasets', 'events such as aws', 'and optimize ai models', 'with staff', 'highimpact g', 'clear and de

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

filtered_skills = []
for skill in unique_skills_CDS:
  skill_words = skill.lower().split()
  if any(word not in stop_words for word in skill_words):
    filtered_skills.append(skill)

unique_skills = filtered_skills
unique_skills


In [ ]:
import pandas as pd
from google.colab import files

# Convert to DataFrame
df_skills = pd.DataFrame(unique_skills, columns=['Extracted competences'])  # change column name as needed

# Save to CSV
df_skills.to_csv('Unique_skills.csv', index=False)

# Download the CSV file
files.download('Unique_skills.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
uploaded = files.upload("Unique_skills.csv")

import io
import pandas as pd

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

  # Assuming the uploaded file is a CSV
  Unique_skills = pd.read_csv(io.BytesIO(uploaded[fn]))

  # Now you can work with the DataFrame 'df'
  print(Unique_skills.head())

Saving Unique_skills.csv to Unique_skills.csv/Unique_skills.csv
User uploaded file "Unique_skills.csv/Unique_skills.csv" with length 309099 bytes
                               Extracted competences
0                               ##s research efforts
1  highly complex issues escalated by direct repo...
2                               responsible ai usage
3                                 and support forest
4                                   conferences, and


In [ ]:
#Creation of reference set and embeddings
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd

# Load the extended reference set
uploaded = files.upload("extended_refset.csv")

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

  # Assuming the uploaded file is a CSV
  Reference_set = pd.read_csv(io.BytesIO(uploaded[fn]))

# Flatten and embed the reference competences
flattened_ref_set = [(category, competence) for category, competences in Reference_set.items() for competence in competences]
model = SentenceTransformer('all-MiniLM-L6-v2')
competences = [comp[1] for comp in flattened_ref_set]
ref_embeddings = model.encode(competences)

# Save the embeddings and structure
np.save("reference_embeddings.npy", ref_embeddings)
reference_df = pd.DataFrame(flattened_ref_set, columns=["Category", "Competence"])
reference_df["Embedding"] = list(ref_embeddings)
reference_df.to_csv("reference_set.csv", index=False)

print("Reference set and embeddings saved.")

Saving extended_refset.csv to Extended_refset.csv/extended_refset.csv
User uploaded file "Extended_refset.csv/extended_refset.csv" with length 36726 bytes
Reference set and embeddings saved.


In [ ]:
Unique_skills

,Extracted competences
0,##s research efforts
1,highly complex issues escalated by direct repo...
2,responsible ai usage
3,and support forest
4,"conferences, and"
...,...
11352,as an ambassador for the organization
11353,and deploy generative ai g
11354,seek
11355,incubating


In [ ]:
#Compute cosine similarity between extracted skills and competences in references set
from sklearn.metrics.pairwise import cosine_similarity

#Extract the skills from Unique skills df
skills = Unique_skills["Extracted competences"].tolist()

# Load reference embeddings
ref_embeddings = np.load("reference_embeddings.npy")

# Embed your skills
skill_embeddings = model.encode(skills)

# Compute cosine similarity
similarity_matrix = cosine_similarity(skill_embeddings, ref_embeddings)

# Output results
for i, skill in enumerate(skills):
    most_similar_idx = similarity_matrix[i].argmax()
    print(f"Skill: {skills}")
    print(f"Closest Competence: {flattened_ref_set[most_similar_idx][1]} (Category: {flattened_ref_set[most_similar_idx][0]})")
    print(f"Similarity: {similarity_matrix[i].max():.4f}")
    print()